<a href="https://colab.research.google.com/github/YBen1/Analyse_Flight_Data/blob/main/analysing_flight_delay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Data
Load all the data for the Year 2019 https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr

In [ ]:
!python -m pip install "dask[complete]"
!pip install graphviz

In [ ]:
import numpy as np

import pandas as pd

import dask

import dask.dataframe as dd

import dask.array as da

import dask.bag as db
import dask.dataframe as dd

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

##Load Data

In [ ]:
first_time_running = True
save_as_csv = False #can only be saved if not existing - if it allready exist, should be false 'False' 

In [ ]:
data_def = {'Div1Airport' : 'object', 'Div1TailNum': 'object', 'Div2Airport' : 'object', 'Div2TailNum' : 'object', 'Div3Airport': 'object', 'ArrTime': 'float64',
       'CancellationCode': 'object',
       'DepTime': 'float64',
       'DepartureDelayGroups': 'float64',
       'WheelsOff': 'float64',
       'WheelsOn': 'float64','Div3TailNum': 'object' }

In [ ]:
def load_data(first_time_running, save_as_csv):
  if first_time_running == True:
    table = dd.read_csv('/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_1.csv',  
                        dtype = data_def,low_memory=False)
    table.shape
    for month in range(2,13): 
      month_str = str(month)
      path_name = '/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_'+ month_str +'.csv'
      table_part = dd.read_csv(path_name,dtype = data_def, low_memory=False)
      table = dd.concat([table,table_part])
      print(month_str +'/12')
    if save_as_csv == True:  
      print("saving DataFrame as csv can take a few min...")  
      table_pd = pd.read_csv('/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_1.csv'
          ,low_memory=False)
      for month in range(2,13): 
        month_str = str(month)
        path_name = '/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_'+ month_str +'.csv'
        table_pd_part = pd.read_csv(path_name, low_memory=False)
        table_pd = pd.concat([table_pd,table_pd_part])
        print(month_str +'/12')  
        table_pd.to_csv('/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_2019_all.csv')  
  else:
    table = dd.read_csv('/content/drive/MyDrive/Analys_Flight_Data/data/On_Time_Reporting_Carrier_On_Time_Performance_2019_all.csv',
                        dtype = data_def,low_memory=False)
  
  #table = table.dropna(how = 'all') #Werfe alle Spalten mit "NaN" raus, da keien Information enthalten
  return table  


In [ ]:
flights_table = load_data(first_time_running, save_as_csv)

Let see arriving delay per airport

In [ ]:
flights_table.describe()

In [ ]:
US_2019_Airport_Delay = flights_table.groupby(['Reporting_Airline'])['ArrDelayMinutes'].mean()
US_2019_Airport_Delay.compute().plot(kind="bar", figsize=(20, 9))

In [ ]:
list(flights_table.columns)

In [ ]:
flights_table.loc[flights_table.ArrDelay > 20][['ArrDelay','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay', 'LateAircraftDelay']].head()

In [ ]:
#flights_from_SA_df = flights_table.loc[flights_table.OriginCityName == 'San Francisco, CA'] # DataFrame mit fluegen aus San Francisco
flights_to_LAX_df = flights_table.loc[flights_table.Dest== 'LAX'][['FlightDate','Reporting_Airline','OriginCityName','Origin','ArrDelayMinutes','DepDelayMinutes','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay', 'LateAircraftDelay']].replace(np.nan, 0) # DataFrame mit fluegen nach San Francisco
to_LAX_with_dalay = flights_to_LAX_df.loc[flights_table.ArrDelayMinutes > 0] #DataFrame mit Fluegen die Versaetungen hatten
to_LAX_with_small_delay = to_LAX_with_dalay.loc[flights_table.ArrDelayMinutes< 5]

#Questions:
-how many lights have been there from LAX? \\
-how many had a delay? \\

#Fragen Airline spezifisch:
-welche Airlines hatten im Mittel die längsten Verspätungen? \\
-wie Teilen sich die Verspätungsgründe auf?

In [ ]:
#Allgemein
num_to_LAX =  len(flights_to_LAX_df.index) #

num_daley_to_LAX = len(to_LAX_with_dalay.index)

num_no_dealy_to_LAX = num_to_LAX -num_daley_to_LAX

num_small_dealy  = len(to_LAX_with_small_delay.index)

In [ ]:
print(str(num_to_LAX) + ' flights arrived at SF, ' + str(num_no_dealy_to_LAX) + ' on time and ' + str(num_daley_to_LAX) +' with a delay')

In [ ]:
delay_mean =  flights_to_LAX_df.ArrDelayMinutes.mean()

delay_std = flights_to_LAX_df.ArrDelayMinutes.std()

In [ ]:
flights_to_LAX_df.ArrDelayMinutes.compute().describe()

In [ ]:
%%time

#airline_delay_mean = flights_to_SF_df.groupby('Reporting_Airline')['DepDelayMinutes','ArrDelayMinutes'].mean()

airline_delay_mean = flights_to_LAX_df.groupby(['Reporting_Airline']).mean()
airline_delay_std = flights_to_LAX_df.groupby(['Reporting_Airline']).std(ddof = 0)

city_delay_mean = flights_to_LAX_df.groupby(['OriginCityName','Origin']).mean() #[['ArrDelayMinutes', 'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']]
city_delay_std = flights_to_LAX_df.groupby(['OriginCityName','Origin']).std(ddof = 0)

airline_city_delay_mean = flights_to_LAX_df.groupby(['Reporting_Airline','Origin']).mean()
airline_city_delay_std = flights_to_LAX_df.groupby(['Reporting_Airline','Origin']).std(ddof = 0)


This Step can take a few Minuts.. depanding on your computer.


In [ ]:
%%time

pd_airline_delay_mean, pd_airline_delay_std = dask.compute(airline_delay_mean, airline_delay_std)

pd_city_delay_mean, pd_city_delay_std = dask.compute(city_delay_mean, city_delay_std)
    
pd_airline_city_delay_mean, pd_airline_city_delay_std = dask.compute(airline_city_delay_mean, airline_city_delay_std)


In [ ]:
%%time
delay_mean_res, delay_std_res  = dask.compute(delay_mean, delay_std)

In [ ]:
airline_delay_std.head()

In [ ]:
%%time
#delay_mean_res, delay_std_res, airline_delay_mean_res, airline_delay_std_res = dask.compute(delay_mean, delay_std, airline_delay_mean, airline_delay_std )

print(delay_mean_res)
print(delay_std_res)

In [ ]:
pd_airline_city_delay_mean.head()

In [ ]:
pd_airline_delay_mean[['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']].plot(kind="bar", figsize=(20, 9))
pd_city_delay_mean[['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']].plot(kind="bar", figsize=(30, 10))
pd_airline_delay_std[['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']].plot(kind="bar", figsize=(20, 9))

How are these categories defined?

####Air Carrier:
The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.).
####Extreme Weather:
 Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.
####National Aviation System (NAS): 
Delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.
####Late-arriving aircraft: 
A previous flight with same aircraft arrived late, causing the present flight to depart late.
####Security: 
Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.


In [ ]:
df_airline_delay = pd.merge(pd_airline_delay_mean, pd_airline_delay_std, on='Reporting_Airline', suffixes=('_mean', '_std'))
df_airline_delay

In [ ]:
df_city_delay = pd.merge(pd_city_delay_mean, pd_city_delay_std , on='OriginCityName', suffixes=('_mean', '_std'))
df_city_delay

In [ ]:
%matplotlib inline
#flights_to_SFO_df.groupby(['FlightDate','Reporting_Airline']).mean()['ArrDelayMinutes'].compute().unstack().plot(figsize=(30, 10))


In [ ]:
flights_to_LAX_df.loc[flights_to_LAX_df.Reporting_Airline== 'AA'].groupby(['FlightDate'])[['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay', 'LateAircraftDelay']].mean().compute().plot(figsize=(30, 10))

In [ ]:
flights_to_LAX_df.loc[flights_to_LAX_df.Reporting_Airline== 'AA'].groupby(['FlightDate'])[['CarrierDelay','NASDelay','WeatherDelay']].mean().compute().plot(figsize=(30, 10))